# Getting Started with Miyagi's Retrieval Augmented Generation (RaG) Workflow using Azure Cognitive Search (ACS) and Semantic Kernel

To quickly get started, follow these steps:

1. Ensure the [Polyglot notebooks extension](https://marketplace.visualstudio.com/items?itemName=ms-dotnettools.dotnet-interactive-vscode) is installed.
2. [Create a new Azure OpenAI service (or use an existing OpenAI service)](https://learn.microsoft.com/en-us/azure/ai-services/openai/chatgpt-quickstart?tabs=command-line&pivots=programming-language-studio#prerequisites).
3. [Deploy the `gpt-35-turbo` and `text-embeddings-ada-002` models](https://learn.microsoft.com/en-us/azure/ai-services/openai/concepts/models#working-with-models).
4. [Create an Azure Cognitive Search instance and enable the Semantic Search capability](https://learn.microsoft.com/en-us/azure/search/semantic-search-overview#enable-semantic-search).
5. Copy the `.env.example` file from the `rag` folder to `dotnet/.env` and paste the corresponding values from the resources you provisioned in the earlier steps.
6. Click on `Run All`.

> You will need an [.Net 7 SDK](https://dotnet.microsoft.com/en-us/download) and [Polyglot](https://marketplace.visualstudio.com/items?itemName=ms-dotnettools.dotnet-interactive-vscode) to get started with this notebook using .Net Interactive

## Overview
In this notebook, we will show you how to quickly get started with Miyagi's Retrieval Augmented Generation (RaG) workflow using Semantic Kernel, Azure Cognitive Search (ACS) and Cosmos. Through this use case, involving stock recommendations, you will understand the basics of RaG and the capabilities of Semantic Kernel, Azure Cognitive Search and Cosmos to adapt to your specific use case.

## Load settings from .env file

In [1]:
#r "nuget: dotenv.net, 3.1.2"
dotenv.net.DotEnv.Load();
var env = dotenv.net.DotEnv.Read();

Installed Packages dotenv.net, 3.1.2

## Prepare the kernel with memory using Azure Cognitive Search

We first add the nuget packages (dependecies) required to instantiate the kernel. 

In [2]:
#r "nuget: Microsoft.SemanticKernel, 1.0.0-beta1"
#r "nuget: Microsoft.SemanticKernel.Connectors.Memory.AzureCognitiveSearch, 1.0.0-beta1"

Installed Packages Microsoft.SemanticKernel, 1.0.0-beta1 Microsoft.SemanticKernel.Connectors.Memory.AzureCognitiveSearch, 1.0.0-beta1

In [3]:
using Microsoft.SemanticKernel;
using Microsoft.SemanticKernel.Connectors.Memory.AzureCognitiveSearch;
using Microsoft.SemanticKernel.Connectors.AI.OpenAI;
using Microsoft.SemanticKernel.Memory;
using Microsoft.SemanticKernel.Plugins.Memory;
using Microsoft.SemanticKernel.Text;
using Microsoft.Extensions.Logging;
using Microsoft.Extensions.Logging.Abstractions;
using System.IO;
using System.Text.Json;

With dependencies installed and configured, we create an instance of the kernel using the built-in `Kernel.Builder` class

In [4]:
ILoggerFactory miyagiLoggerFactory = NullLoggerFactory.Instance;
var kernel = Kernel.Builder
    .WithLoggerFactory(miyagiLoggerFactory)

    .WithAzureChatCompletionService(
        env["AZURE_OPENAI_CHAT_MODEL"],
        env["AZURE_OPENAI_ENDPOINT"],
        env["AZURE_OPENAI_API_KEY"]
    )
        
    .Build();

To make the kernel stateful, we will use Memory Plugin with a Azure Cognitive Search and Embedding backend. 

In [5]:
var memoryBuilder = new MemoryBuilder();
memoryBuilder
    .WithAzureTextEmbeddingGenerationService(
        env["AZURE_OPENAI_EMBEDDING_MODEL"],
        env["AZURE_OPENAI_ENDPOINT"],
        env["AZURE_OPENAI_API_KEY"]
    )

    .WithMemoryStore(
        new AzureCognitiveSearchMemoryStore(
            env["AZURE_COGNITIVE_SEARCH_ENDPOINT"],
            env["AZURE_COGNITIVE_SEARCH_API_KEY"]
        ));

var memory = memoryBuilder.Build();

## Vectorize and persist embeddings in Azure Cognitive Search with Semantic Kernel

Embeddings are a type of data transformation that convert text into high-dimensional vectors that capture semantic relationships between words or text chunks. This vectorized form allows machines to process text in a manner akin to numerical data, enabling operations like similarity comparison which is foundational for vector databases to retrieve meaningful context for LLMs. Embeddings are essential to generate and retrieve relevant content in a RaG workflow because:

1. They normalize your data in a common mathematical representation, making it machine-friendly
1. They compress data, retaining essential representations while reducing dimensionality
1. They preserve relationships within your data
1. They are representationally dense, meaning they are more efficient to store and process


> See a visual representation of the embeddings in 2D space below (from Cohere):

![Semantic Search](https://txt.cohere.com/content/images/2023/01/Vis-7-2.jpg)

Now, let’s delve into the process of creating and utilizing embeddings in the context of ACS.

In this scenario, we have a snippet of proprietary text data—extracted from a book—which needs to be processed and indexed for semantic search. The steps involve:

1. Chunking the Text: Breaking down the text into manageable pieces, referred to as 'chunks'.
1. Generating Embeddings: Transforming these chunks into high-dimensional vectors using an Azure Text Embedding Generation Service.
1. Persisting Embeddings to ACS: Storing these embeddings in Azure Cognitive Search to facilitate semantic search.

The provided code snippet orchestrates these steps. It reads the text data from a local file, chunks it, generates embeddings for each chunk, and then persists these embeddings to ACS. The specific operations are abstracted for simplicity using methods like SaveInformationAsync, which is part of a hypothetical kernel object in this code. The end result is a collection of indexed embeddings in ACS that significantly enhance search quality by moving beyond mere keyword matching to understanding semantic relationships.

In [6]:
var dataset = "intelligent-investor.txt";
var recommendationServicePath = "../../../../services/recommendation-service/dotnet";
const int MaxTokensPerParagraph = 160;
const int MaxTokensPerLine = 60;

// Load text data from a local file
var filePath = Path.Combine(recommendationServicePath, "resources", "sample-datasets", dataset);
var streamReader = new StreamReader(filePath);
var text = await streamReader.ReadToEndAsync();

// Define a memory collection name for indexing
var memoryCollectionName = "userId";

// Chunk the text into lines and paragraphs for manageable processing
var lines = TextChunker.SplitPlainTextLines(text, MaxTokensPerLine);
var chunks = TextChunker.SplitPlainTextParagraphs(lines, MaxTokensPerParagraph);

// Loop through each chunk to generate embeddings and persist to ACS
for (var i = 0; i < chunks.Count; i++)
{
    var chunk = chunks[i];
    // Generate an identifier for each chunk
    var key = await memory.SaveInformationAsync(
        memoryCollectionName,
        chunk,
        $"{dataset}-{i}",
        $"Dataset: {dataset} Chunk: {i}",
        i.ToString());
}

// Output the result to console
System.Console.WriteLine($"Saved {chunks.Count} chunks to memory collection {memoryCollectionName}");


Saved 46 chunks to memory collection userId


## Search and retrieve documents using Semantic Kernel

To search and retrieve documents, we will use the `SearchAsync` method of Semantic memory that encapsulates retrieval abstractions of different memory providers. This method takes in a query and returns a list of documents that are relevant to the query.

In [7]:
var query = "Ben Graham's investment philosophy";
Console.WriteLine(query + "\n");

var results = memory.SearchAsync(collection: memoryCollectionName, query, limit: 2);
await foreach(var result in results)
{
    Console.WriteLine("   " + result.Metadata.Text);
    Console.WriteLine("   Relevance: " + result.Relevance + "\n");
}

Ben Graham's investment philosophy

   Graham designed his approach to craft a reliable portfolio that requires minimum maintenance and offers maximum odds of a steady return.
By sticking to a formula for investment decisions, the defensive investor gives up the risk of speculating on stock movements and gains steady returns.
After the initial curation, if the investor trades more than twice a year, it is a clear sign something has gone wrong.
[widget]
SUMMARY
Price and value are two different concepts, and stock prices frequently don't reflect a company's actual value. More risk does not necessarily correlate with more gain.
   Relevance: 0.8651936274721188

   Projection is dangerous because the future is uncertain, and inflation, economic recessions, pandemics and geopolitical upheavals often arrive without warning.
Graham argues that it is a fool's errand for an ordinary investor to attempt making money by timing the market.
Graham advocates for a protection-based approach that doe

## Grounding Miyagi prompts with SK's Memory "recall"

> Note that this prompt template (semantic function) is located under services/reccommendation-service/dotnet/plugins/AdvisorPlugin

In [8]:
using Microsoft.SemanticKernel.SemanticFunctions;

// recall is from the TextMemoryPlugin, which has a recall method that can be invoked from Semantic Function to perform retrieval
kernel.ImportFunctions(new TextMemoryPlugin(memory));

var pluginFolder = $"{recommendationServicePath}/plugins";
var advisorPlugin = kernel.ImportSemanticFunctionsFromDirectory(pluginFolder, "AdvisorPlugin");
advisorPlugin

key value InvestmentAdvise {"Name":"InvestmentAdvise","PluginName":"AdvisorPlugin","Description":"Gives financial advise on how to allocate portfolio, given a risk tolerance and a set of stocks","RequestSettings":{"service_id":null,"model_id":null,"max_tokens":512,"temperature":0.9,"top_p":1.0,"presence_penalty":1.0,"frequenc... Name InvestmentAdvise PluginName AdvisorPlugin Description Gives financial advise on how to allocate portfolio, given a risk tolerance and a set of stocks RequestSettings Microsoft.SemanticKernel.AI.AIRequestSettings ServiceId <null> ModelId <null> ExtensionData key value max_tokens 512 temperature 0.9 top_p 1 presence_penalty 1 frequency_penalty 1 Parameters index value 0 ParameterView { Name = portfolio, Description = Asset portfolio as JSON text, DefaultValue = , Type = , IsRequired = } Name portfolio Description Asset portfolio as JSON text DefaultValue Type <null> IsRequired <null> 1 ParameterView { Name = user, Description = User information as JSON text, DefaultValue = , Type = , IsRequired = } Name user Description User information as JSON text DefaultValue Type <null> IsRequired <null> _promptTemplate Microsoft.SemanticKernel.SemanticFunctions.PromptTemplate Parameters index value 0 ParameterView { Name = portfolio, Description = Asset portfolio as JSON text, DefaultValue = , Type = , IsRequired = } Name portfolio Description Asset portfolio as JSON text DefaultValue Type <null> IsRequired <null> 1 ParameterView { Name = user, Description = User information as JSON text, DefaultValue = , Type = , IsRequired = } Name user Description User information as JSON text DefaultValue Type <null> IsRequired <null> SkillName AdvisorPlugin IsSemantic True PortfolioAllocation {"Name":"PortfolioAllocation","PluginName":"AdvisorPlugin","Description":"Gives financial advise on how to allocate assets, given a risk tolerance","RequestSettings":{"service_id":null,"model_id":null,"max_tokens":512,"temperature":0.8,"top_p":1.0,"presence_penalty":0.0,"frequency_penalty":0.0},"Par... Name PortfolioAllocation PluginName AdvisorPlugin Description Gives financial advise on how to allocate assets, given a risk tolerance RequestSettings Microsoft.SemanticKernel.AI.AIRequestSettings ServiceId <null> ModelId <null> ExtensionData key value max_tokens 512 temperature 0.8 top_p 1 presence_penalty 0 frequency_penalty 0 Parameters index value 0 ParameterView { Name = portfolio, Description = Asset portfolio as JSON text, DefaultValue = , Type = , IsRequired = } Name portfolio Description Asset portfolio as JSON text DefaultValue Type <null> IsRequired <null> 1 ParameterView { Name = user, Description = User information as JSON text, DefaultValue = , Type = , IsRequired = } Name user Description User information as JSON text DefaultValue Type <null> IsRequired <null> _promptTemplate Microsoft.SemanticKernel.SemanticFunctions.PromptTemplate Parameters index value 0 ParameterView { Name = portfolio, Description = Asset portfolio as JSON text, DefaultValue = , Type = , IsRequired = } Name portfolio Description Asset portfolio as JSON text DefaultValue Type <null> IsRequired <null> 1 ParameterView { Name = user, Description = User information as JSON text, DefaultValue = , Type = , IsRequired = } Name user Description User information as JSON text DefaultValue Type <null> IsRequired <null> SkillName AdvisorPlugin IsSemantic True

### Set context variables

In [9]:
var context = kernel.CreateNewContext();

// Set the parameters for the TextMemorySkill
context.Variables[TextMemoryPlugin.CollectionParam] = memoryCollectionName;
context.Variables[TextMemoryPlugin.RelevanceParam] = "0.8";
context.Variables[TextMemoryPlugin.LimitParam] = "3";

// Set the parameters for the AdvisorPlugin
var stocks = new[] {
    new {symbol = "MSFT", allocation = 0.3},
    new {symbol = "ACN", allocation = 0.1},
    new {symbol = "JPM", allocation = 0.3},
    new {symbol = "PEP", allocation = 0.3}
};
context.Variables["stocks"] = JsonSerializer.Serialize(stocks);

context.Variables["userId"] = "50";
context.Variables["voice"] = "Jim Cramer";
context.Variables["risk"] = "aggressive";

context

Result ModelResults (empty) Culture en Parent Parent Parent Parent Parent LCID 127 KeyboardLayoutId 127 Name IetfLanguageTag DisplayName Invariant Language (Invariant Country) NativeName Invariant Language (Invariant Country) EnglishName Invariant Language (Invariant Country) TwoLetterISOLanguageName iv ThreeLetterISOLanguageName ivl ThreeLetterWindowsLanguageName IVL CompareInfo CompareInfo - TextInfo TextInfo - IsNeutralCulture False CultureTypes SpecificCultures, InstalledWin32Cultures NumberFormat System.Globalization.NumberFormatInfo DateTimeFormat System.Globalization.DateTimeFormatInfo Calendar System.Globalization.GregorianCalendar OptionalCalendars [ System.Globalization.GregorianCalendar ] UseUserOverride False IsReadOnly True LCID 127 KeyboardLayoutId 127 Name IetfLanguageTag DisplayName Invariant Language (Invariant Country) NativeName Invariant Language (Invariant Country) EnglishName Invariant Language (Invariant Country) TwoLetterISOLanguageName iv ThreeLetterISOLanguageName ivl ThreeLetterWindowsLanguageName IVL CompareInfo CompareInfo - Name Version System.Globalization.SortVersion LCID 127 TextInfo TextInfo - ANSICodePage 1252 OEMCodePage 437 MacCodePage 10000 EBCDICCodePage 37 LCID 127 CultureName IsReadOnly True ListSeparator , IsRightToLeft False IsNeutralCulture False CultureTypes SpecificCultures, InstalledWin32Cultures NumberFormat System.Globalization.NumberFormatInfo CurrencyDecimalDigits 2 CurrencyDecimalSeparator . IsReadOnly True CurrencyGroupSizes [ 3 ] NumberGroupSizes [ 3 ] PercentGroupSizes [ 3 ] CurrencyGroupSeparator , CurrencySymbol ¤ NaNSymbol NaN CurrencyNegativePattern 0 NumberNegativePattern 1 PercentPositivePattern 0 PercentNegativePattern 0 NegativeInfinitySymbol -Infinity NegativeSign - NumberDecimalDigits 2 NumberDecimalSeparator . NumberGroupSeparator , CurrencyPositivePattern 0 PositiveInfinitySymbol Infinity PositiveSign + PercentDecimalDigits 2 PercentDecimalSeparator . PercentGroupSeparator , PercentSymbol % PerMilleSymbol ‰ NativeDigits [ 0, 1, 2, 3, 4, 5, 6, 7, 8, 9 ] DigitSubstitution None DateTimeFormat System.Globalization.DateTimeFormatInfo AMDesignator AM Calendar System.Globalization.GregorianCalendar DateSeparator / FirstDayOfWeek Sunday CalendarWeekRule FirstDay FullDateTimePattern dddd, dd MMMM yyyy HH:mm:ss LongDatePattern dddd, dd MMMM yyyy LongTimePattern HH:mm:ss MonthDayPattern MMMM dd PMDesignator PM RFC1123Pattern ddd, dd MMM yyyy HH':'mm':'ss 'GMT' ShortDatePattern MM/dd/yyyy ShortTimePattern HH:mm SortableDateTimePattern yyyy'-'MM'-'dd'T'HH':'mm':'ss TimeSeparator : UniversalSortableDateTimePattern yyyy'-'MM'-'dd HH':'mm':'ss'Z' YearMonthPattern yyyy MMMM AbbreviatedDayNames [ Sun, Mon, Tue, Wed, Thu, Fri, Sat ] ShortestDayNames [ Su, Mo, Tu, We, Th, Fr, Sa ] DayNames [ Sunday, Monday, Tuesday, Wednesday, Thursday, Friday, Saturday ] AbbreviatedMonthNames [ Jan, Feb, Mar, Apr, May, Jun, Jul, Aug, Sep, Oct, Nov, Dec, ] MonthNames [ January, February, March, April, May, June, July, August, September, October, November, December, ] IsReadOnly True NativeCalendarName Gregorian Calendar AbbreviatedMonthGenitiveNames [ Jan, Feb, Mar, Apr, May, Jun, Jul, Aug, Sep, Oct, Nov, Dec, ] MonthGenitiveNames [ January, February, March, April, May, June, July, August, September, October, November, December, ] Calendar System.Globalization.GregorianCalendar MinSupportedDateTime 1/1/0001 12:00:00 AM MaxSupportedDateTime 12/31/9999 11:59:59 PM AlgorithmType SolarCalendar CalendarType Localized Eras [ 1 ] TwoDigitYearMax 2029 IsReadOnly True OptionalCalendars index value 0 System.Globalization.GregorianCalendar UseUserOverride False IsReadOnly True LCID 127 KeyboardLayoutId 127 Name IetfLanguageTag DisplayName Invariant Language (Invariant Country) NativeName Invariant Language (Invariant Country) EnglishName Invariant Language (Invariant Country) TwoLetterISOLanguageName iv ThreeLetterISOLanguageName ivl ThreeLetterWindowsLanguageName IVL CompareInfo CompareInfo - Name Version S

### Create native function

In [10]:
using System.ComponentModel;

/// <summary>
///     UserProfilePlugin shows a native skill example to look user info given userId.
/// </summary>
/// <example>
///     Usage: kernel.ImportSkill("UserProfilePlugin", new UserProfilePlugin());
///     Examples:
///     SKContext["userId"] = "000"
///     {{UserProfilePlugin.GetUserAge $userId }} => {userProfile}
/// </example>
public class UserProfilePlugin
{
    /// <summary>
    ///     Name of the context variable used for UserId.
    /// </summary>
    public const string UserId = "UserId";

    private const string DefaultUserId = "40";
    private const int DefaultAnnualHouseholdIncome = 150000;
    private const int Normalize = 81;

    /// <summary>
    ///     Lookup User's age for a given UserId.
    /// </summary>
    /// <example>
    ///     SKContext[UserProfilePlugin.UserId] = "000"
    /// </example>
    /// <param name="context">Contains the context variables.</param>
    [SKFunction]
    [SKName("GetUserAge")]
    [Description("Given a userId, get user age")]
    public string GetUserAge(
        [Description("Unique identifier of a user")]
        string userId)
    {
        // userId = context.Variables.ContainsKey(UserId) ? context[UserId] : DefaultUserId;
        userId = string.IsNullOrEmpty(userId) ? DefaultUserId : userId;

        int age;

        if (int.TryParse(userId, out var parsedUserId))
            age = parsedUserId > 100 ? parsedUserId % Normalize : parsedUserId;
        else
            age = int.Parse(DefaultUserId);

        // invoke a service to get the age of the user, given the userId
        return age.ToString();
    }

    /// <summary>
    ///     Lookup User's annual income given UserId.
    /// </summary>
    /// <example>
    ///     SKContext[UserProfilePlugin.UserId] = "000"
    /// </example>
    /// <param name="context">Contains the context variables.</param>
    [SKFunction]
    [SKName("GetAnnualHouseholdIncome")]
    [Description("Given a userId, get user annual household income")]
    public string GetAnnualHouseholdIncome(
        [Description("Unique identifier of a user")]
        string userId)
    {
        // userId = context.Variables.ContainsKey(UserId) ? context[UserId] : DefaultUserId;
        userId = string.IsNullOrEmpty(userId) ? DefaultUserId : userId;

        var random = new Random();
        var randomMultiplier = random.Next(1000, 8000);

        // invoke a service to get the annual household income of the user, given the userId
        var annualHouseholdIncome = int.TryParse(userId, out var parsedUserId)
            ? parsedUserId * randomMultiplier
            : DefaultAnnualHouseholdIncome;

        return annualHouseholdIncome.ToString();
    }
}

In [11]:
// import the UserProfilePlugin
kernel.ImportFunctions(new UserProfilePlugin(), "UserProfilePlugin");

### Invoke the LLM

In [12]:
using Microsoft.SemanticKernel.TemplateEngine.Basic;

var promptRenderer = new BasicPromptTemplateEngine();
var renderedPrompt = await promptRenderer.RenderAsync(advisorPlugin["InvestmentAdvise"].ToString(), context);
Console.WriteLine(renderedPrompt);

{"Name":"InvestmentAdvise","PluginName":"AdvisorPlugin","Description":"Gives financial advise on how to allocate portfolio, given a risk tolerance and a set of stocks","RequestSettings":{"service_id":null,"model_id":null,"max_tokens":512,"temperature":0.9,"top_p":1.0,"presence_penalty":1.0,"frequency_penalty":1.0},"Parameters":[{"Name":"portfolio","Description":"Asset portfolio as JSON text","DefaultValue":"","Type":null,"IsRequired":null},{"Name":"user","Description":"User information as JSON text","DefaultValue":"","Type":null,"IsRequired":null}],"_promptTemplate":{"Parameters":[{"Name":"portfolio","Description":"Asset portfolio as JSON text","DefaultValue":"","Type":null,"IsRequired":null},{"Name":"user","Description":"User information as JSON text","DefaultValue":"","Type":null,"IsRequired":null}]},"SkillName":"AdvisorPlugin","IsSemantic":true}


In [17]:
context

Result ModelResults (empty) Culture en Parent Parent Parent Parent Parent LCID 127 KeyboardLayoutId 127 Name IetfLanguageTag DisplayName Invariant Language (Invariant Country) NativeName Invariant Language (Invariant Country) EnglishName Invariant Language (Invariant Country) TwoLetterISOLanguageName iv ThreeLetterISOLanguageName ivl ThreeLetterWindowsLanguageName IVL CompareInfo CompareInfo - TextInfo TextInfo - IsNeutralCulture False CultureTypes SpecificCultures, InstalledWin32Cultures NumberFormat System.Globalization.NumberFormatInfo DateTimeFormat System.Globalization.DateTimeFormatInfo Calendar System.Globalization.GregorianCalendar OptionalCalendars [ System.Globalization.GregorianCalendar ] UseUserOverride False IsReadOnly True LCID 127 KeyboardLayoutId 127 Name IetfLanguageTag DisplayName Invariant Language (Invariant Country) NativeName Invariant Language (Invariant Country) EnglishName Invariant Language (Invariant Country) TwoLetterISOLanguageName iv ThreeLetterISOLanguageName ivl ThreeLetterWindowsLanguageName IVL CompareInfo CompareInfo - Name Version System.Globalization.SortVersion LCID 127 TextInfo TextInfo - ANSICodePage 1252 OEMCodePage 437 MacCodePage 10000 EBCDICCodePage 37 LCID 127 CultureName IsReadOnly True ListSeparator , IsRightToLeft False IsNeutralCulture False CultureTypes SpecificCultures, InstalledWin32Cultures NumberFormat System.Globalization.NumberFormatInfo CurrencyDecimalDigits 2 CurrencyDecimalSeparator . IsReadOnly True CurrencyGroupSizes [ 3 ] NumberGroupSizes [ 3 ] PercentGroupSizes [ 3 ] CurrencyGroupSeparator , CurrencySymbol ¤ NaNSymbol NaN CurrencyNegativePattern 0 NumberNegativePattern 1 PercentPositivePattern 0 PercentNegativePattern 0 NegativeInfinitySymbol -Infinity NegativeSign - NumberDecimalDigits 2 NumberDecimalSeparator . NumberGroupSeparator , CurrencyPositivePattern 0 PositiveInfinitySymbol Infinity PositiveSign + PercentDecimalDigits 2 PercentDecimalSeparator . PercentGroupSeparator , PercentSymbol % PerMilleSymbol ‰ NativeDigits [ 0, 1, 2, 3, 4, 5, 6, 7, 8, 9 ] DigitSubstitution None DateTimeFormat System.Globalization.DateTimeFormatInfo AMDesignator AM Calendar System.Globalization.GregorianCalendar DateSeparator / FirstDayOfWeek Sunday CalendarWeekRule FirstDay FullDateTimePattern dddd, dd MMMM yyyy HH:mm:ss LongDatePattern dddd, dd MMMM yyyy LongTimePattern HH:mm:ss MonthDayPattern MMMM dd PMDesignator PM RFC1123Pattern ddd, dd MMM yyyy HH':'mm':'ss 'GMT' ShortDatePattern MM/dd/yyyy ShortTimePattern HH:mm SortableDateTimePattern yyyy'-'MM'-'dd'T'HH':'mm':'ss TimeSeparator : UniversalSortableDateTimePattern yyyy'-'MM'-'dd HH':'mm':'ss'Z' YearMonthPattern yyyy MMMM AbbreviatedDayNames [ Sun, Mon, Tue, Wed, Thu, Fri, Sat ] ShortestDayNames [ Su, Mo, Tu, We, Th, Fr, Sa ] DayNames [ Sunday, Monday, Tuesday, Wednesday, Thursday, Friday, Saturday ] AbbreviatedMonthNames [ Jan, Feb, Mar, Apr, May, Jun, Jul, Aug, Sep, Oct, Nov, Dec, ] MonthNames [ January, February, March, April, May, June, July, August, September, October, November, December, ] IsReadOnly True NativeCalendarName Gregorian Calendar AbbreviatedMonthGenitiveNames [ Jan, Feb, Mar, Apr, May, Jun, Jul, Aug, Sep, Oct, Nov, Dec, ] MonthGenitiveNames [ January, February, March, April, May, June, July, August, September, October, November, December, ] Calendar System.Globalization.GregorianCalendar MinSupportedDateTime 1/1/0001 12:00:00 AM MaxSupportedDateTime 12/31/9999 11:59:59 PM AlgorithmType SolarCalendar CalendarType Localized Eras [ 1 ] TwoDigitYearMax 2029 IsReadOnly True OptionalCalendars index value 0 System.Globalization.GregorianCalendar UseUserOverride False IsReadOnly True LCID 127 KeyboardLayoutId 127 Name IetfLanguageTag DisplayName Invariant Language (Invariant Country) NativeName Invariant Language (Invariant Country) EnglishName Invariant Language (Invariant Country) TwoLetterISOLanguageName iv ThreeLetterISOLanguageName ivl ThreeLetterWindowsLanguageName IVL CompareInfo CompareInfo - Name Version S

In [20]:
string skPrompt = @"
{{$input}}

Give me the TLDR in 5 words.
";

var textToSummarize = @"
    1) A robot may not injure a human being or, through inaction,
    allow a human being to come to harm.

    2) A robot must obey orders given it by human beings except where
    such orders would conflict with the First Law.

    3) A robot must protect its own existence as long as such protection
    does not conflict with the First or Second Law.
";

var tldrFunction = kernel.CreateSemanticFunction(skPrompt, requestSettings: new OpenAIRequestSettings { MaxTokens = 2000, Temperature = 0.2, TopP = 0.5 });

var summaryResult = await kernel.RunAsync(textToSummarize, tldrFunction);
var summary = summaryResult.GetValue<string>();

Console.WriteLine(summary);

Robots protect, obey, and self-preserve.



In [25]:
var result = await advisorPlugin["PortfolioAllocation"].InvokeAsync(context);

// var result = await kernel.RunAsync(context.Variables, advisorPlugin["PortfolioAllocation"]);

result

Microsoft.SemanticKernel.Orchestration.FunctionResult FunctionName PortfolioAllocation PluginName AdvisorPlugin Metadata key value ModelResults index value 0 Microsoft.SemanticKernel.Orchestration.ModelResult

![RaG Workflow](../../../../assets/images/sk-memory-orchestration.png)